In [1]:
library(tidyr)
library(tidyverse) 
library(knitr)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ dplyr   1.0.10
✔ tibble  3.1.8      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
✔ purrr   0.3.4      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


# Data Cleaning/Import

In [2]:
# Read in CSV file
test_data <- read.csv("2023StateOf.csv")

# Remove second row so column names are preserved
test_data <- test_data[-c(1),]

# Change column names
colnames(test_data)[1] <- "Record_ID"
colnames(test_data)[2] <- "email"

# Convert "Unchecked" and empty values to NA in all columns
checked_columns <- colnames(test_data)[-c(1, 2)]  # Get column names excluding Record_ID and email

for (col in checked_columns) {
  test_data[[col]] <- ifelse(test_data[[col]] %in% c("Unchecked", ""), NA, test_data[[col]])
}

# Remove rows with all missing values - remove rows where all values are NA except for "Record_ID", "email", and "Q219" columns
test_data <- test_data[rowSums(is.na(test_data[, 3:844])) != (844 - 3 + 1), ]

# Find rows with duplicate email
duplicate_data <- test_data[duplicated(test_data$email) | duplicated(test_data$email, fromLast = TRUE), ]

# Merge rows with duplicate email, keeping only non-missing values and updating Record_ID
merged_data <- duplicate_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Combine non-duplicate rows with merged rows
final_data <- test_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Convert "Checked" to 1
final_data[] <- lapply(final_data, function(x) ifelse(x == "Checked", 1, x))

In [3]:
# preview of cleaned data
head(final_data, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
6ap464zdb@mozmail.com,687,Community Based Organization (CBO),NA,New York,14527,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete


In [4]:
final_data[,c("Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13")]

Q7,Q8,Q9,Q10,Q11,Q12,Q13
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,NA,NA
1,NA,NA,NA,NA,NA,NA
1,1,1,1,1,1,NA
1,1,1,1,1,1,NA
NA,NA,NA,NA,NA,NA,1
1,1,1,1,1,1,NA
NA,NA,NA,NA,NA,NA,1
1,1,1,1,1,1,NA
1,NA,1,NA,NA,1,NA


## Number of incomplete and complete responses

In [5]:
# Complete vs incomplete for general data
table(final_data$Q819)


  Complete Incomplete 
       819        295 

# Descriptive Statistics for overall data

## Age counts

In [6]:
# Age
gen_age <- table(final_data$Q162)
gen_age


                18-24                 25-34                 35-44 
                   31                   204                   256 
                45-54                 55-64                    65 
                  262                   226                   103 
Prefer not to respond 
                   23 

## Age percentages

In [7]:
# Age
gen_age_perc <- prop.table(gen_age)*100
gen_age_perc


                18-24                 25-34                 35-44 
             2.805430             18.461538             23.167421 
                45-54                 55-64                    65 
            23.710407             20.452489              9.321267 
Prefer not to respond 
             2.081448 

## Race counts

In [8]:
## Race
gen_race <- table(final_data$Q159)
gen_race


        Asian and/or Asian American           Black or African American 
                                 28                                 273 
                        Multiracial    Native American or Alaska Native 
                                 79                                  13 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  55 
            Prefer to self-describe                  White or Caucasian 
                                 26                                 625 

## Race percentages

In [9]:
# Race
gen_race_perc <- prop.table(gen_race)*100
gen_race_perc


        Asian and/or Asian American           Black or African American 
                          2.5362319                          24.7282609 
                        Multiracial    Native American or Alaska Native 
                          7.1557971                           1.1775362 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4528986                           4.9818841 
            Prefer to self-describe                  White or Caucasian 
                          2.3550725                          56.6123188 

## Hispanic/Latinx counts

In [10]:
## Hispanic/Latinx
gen_hisp <- table(final_data$Q161)
gen_hisp



                   No Prefer not to respond                   Yes 
                  887                    21                   197 

## Hispanic/Latinx percentages

In [11]:
gen_hisp_perc <- prop.table(gen_hisp)*100
gen_hisp_perc


                   No Prefer not to respond                   Yes 
            80.271493              1.900452             17.828054 

## Gender identity counts

In [12]:
genid_counts <- table(final_data$Q157)
genid_counts


                                Female/Woman 
                                         682 
Genderqueer/gender non-confirming/non-binary 
                                          71 
                                    Male/Man 
                                         288 
                       Prefer not to respond 
                                          19 
                     Prefer to self-describe 
                                          11 
                             Transgender man 
                                          20 
                           Transgender woman 
                                          14 

## Gender identity percentages

In [13]:
genid_perc <- prop.table(genid_counts)*100
genid_perc


                                Female/Woman 
                                  61.7194570 
Genderqueer/gender non-confirming/non-binary 
                                   6.4253394 
                                    Male/Man 
                                  26.0633484 
                       Prefer not to respond 
                                   1.7194570 
                     Prefer to self-describe 
                                   0.9954751 
                             Transgender man 
                                   1.8099548 
                           Transgender woman 
                                   1.2669683 

## Type of organization counts

In [14]:
## Org type
gen_org_type <- table(final_data$Q1)
gen_org_type


             Academic hospital/clinic setting 
                                           80 
                        Advocacy organization 
                                            5 
              AIDS Service Organization (ASO) 
                                          133 
           Community Based Organization (CBO) 
                                          129 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                          105 
               Health department (non-clinic) 
                                           41 
Health department clinic/public health clinic 
                                          146 
         LBGTQ Health Center/Community Center 
                                           55 
                      Non-profit organization 
                                          159 
                                        Other 
            

## Type of organization percentages

In [15]:
gen_org_perc <- prop.table(gen_org_type)*100
gen_org_perc


             Academic hospital/clinic setting 
                                    7.1813285 
                        Advocacy organization 
                                    0.4488330 
              AIDS Service Organization (ASO) 
                                   11.9389587 
           Community Based Organization (CBO) 
                                   11.5798923 
     Federally Qualified Health Center (FQHC) 
                                   12.0287253 
    Government entity (Local, State, Federal) 
                                    9.4254937 
               Health department (non-clinic) 
                                    3.6804309 
Health department clinic/public health clinic 
                                   13.1059246 
         LBGTQ Health Center/Community Center 
                                    4.9371634 
                      Non-profit organization 
                                   14.2728905 
                                        Other 
            

## State counts

In [16]:
## States
gen_state_counts <- table(final_data$Q3)
gen_state_counts


             Alabama               Alaska              Arizona 
                   8                    1                   15 
            Arkansas           California             Colorado 
                   8                  105                   16 
         Connecticut             Delaware District of Columbia 
                  20                   12                   39 
             Florida              Georgia               Hawaii 
                  64                   26                   11 
               Idaho             Illinois              Indiana 
                   2                   38                   46 
                Iowa               Kansas             Kentucky 
                  13                    4                   30 
           Louisiana                Maine             Maryland 
                  24                    1                   67 
       Massachusetts             Michigan            Minnesota 
                   7                   

## State percentages

In [17]:
gen_state_perc <- prop.table(gen_state_counts)*100
gen_state_perc


             Alabama               Alaska              Arizona 
          0.72072072           0.09009009           1.35135135 
            Arkansas           California             Colorado 
          0.72072072           9.45945946           1.44144144 
         Connecticut             Delaware District of Columbia 
          1.80180180           1.08108108           3.51351351 
             Florida              Georgia               Hawaii 
          5.76576577           2.34234234           0.99099099 
               Idaho             Illinois              Indiana 
          0.18018018           3.42342342           4.14414414 
                Iowa               Kansas             Kentucky 
          1.17117117           0.36036036           2.70270270 
           Louisiana                Maine             Maryland 
          2.16216216           0.09009009           6.03603604 
       Massachusetts             Michigan            Minnesota 
          0.63063063           3.873873

## Geographic area counts

In [18]:
geog_counts <- table(final_data$Q5)
geog_counts


   Rural Suburban   Tribal    Urban 
     228      195        3      686 

## Geographic area percentages

In [19]:
geog_perc <- prop.table(geog_counts)*100
geog_perc


     Rural   Suburban     Tribal      Urban 
20.5035971 17.5359712  0.2697842 61.6906475 

## Role type counts - Clinical vs nonclinical

In [20]:
role_type_counts <- table(final_data$Q65)
role_type_counts


    Clinical Non-clinical 
         380          728 

In [21]:
role_type_perc <- prop.table(role_type_counts)*100
role_type_perc


    Clinical Non-clinical 
    34.29603     65.70397 

## Clinical role counts

In [22]:
clinic_role_counts <- table(final_data$Q66)
clinic_role_counts


                             Dentist               Dietician/Nutritionist 
                                   2                                    1 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                                   1                                   50 
            Licensed Practical Nurse                    Medical Assistant 
                                  10                                   10 
          Mental Health Case Manager              Mental Health Counselor 
                                   5                                    1 
          Mental Health Professional                        Nurse Manager 
                                  16                                   19 
                       Nurse Midwife                   Nurse Practitioner 
                                   1                                   69 
Other Provider/Clinical professional                           Pharmacist 
                        

## Clinical role percentages

In [23]:
clinic_role_perc <- prop.table(clinic_role_counts) * 100 
clinic_role_perc


                             Dentist               Dietician/Nutritionist 
                           0.5277045                            0.2638522 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                           0.2638522                           13.1926121 
            Licensed Practical Nurse                    Medical Assistant 
                           2.6385224                            2.6385224 
          Mental Health Case Manager              Mental Health Counselor 
                           1.3192612                            0.2638522 
          Mental Health Professional                        Nurse Manager 
                           4.2216359                            5.0131926 
                       Nurse Midwife                   Nurse Practitioner 
                           0.2638522                           18.2058047 
Other Provider/Clinical professional                           Pharmacist 
                        

## Non-clinical role counts 

In [24]:
nonclinic_role_counts <- table(final_data$Q71)
nonclinic_role_counts


                         Administrator                               Advocate 
                                   169                                     18 
                             Caregiver      Case Manager/Medical case manager 
                                     1                                    129 
       Clergy/Faith-Based Professional                Community Health Worker 
                                     1                                     64 
       Disease Intervention Specialist                 Finance/Fiscal Manager 
                                    37                                      4 
   Harm Reductionist/Risk Reductionist              Health and Wellness Coach 
                                    32                                      1 
           Health Education Specialist                       Health Navigator 
                                    56                                     36 
               HIV Linkage Coordinator             

## Non-clinical role percentages 

In [25]:
nonclinic_role_perc <- prop.table(nonclinic_role_counts)*100
nonclinic_role_perc


                         Administrator                               Advocate 
                            23.2782369                              2.4793388 
                             Caregiver      Case Manager/Medical case manager 
                             0.1377410                             17.7685950 
       Clergy/Faith-Based Professional                Community Health Worker 
                             0.1377410                              8.8154270 
       Disease Intervention Specialist                 Finance/Fiscal Manager 
                             5.0964187                              0.5509642 
   Harm Reductionist/Risk Reductionist              Health and Wellness Coach 
                             4.4077135                              0.1377410 
           Health Education Specialist                       Health Navigator 
                             7.7134986                              4.9586777 
               HIV Linkage Coordinator             

## Years in role counts

In [26]:
years_counts <- table(final_data$Q94)
years_counts


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        292         166         237         178         235 

## Years in role counts

In [27]:
years_perc <- prop.table(years_counts)*100
years_perc


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   26.35379    14.98195    21.38989    16.06498    21.20939 

## Prescriber counts

In [28]:
prescriber_counts <- table(final_data$Q70)
prescriber_counts


 No Yes 
229 154 

## Prescriber percentage

In [29]:
prescriber_perc <- prop.table(prescriber_counts)*100
prescriber_perc


      No      Yes 
59.79112 40.20888 

## Onsite pharmacy counts

In [30]:
onsite_pharm_counts <- table(final_data$Q6)
onsite_pharm_counts


 No Yes 
681 430 

## Onsite pharmacy perceentages

In [31]:
onsite_pharm_perc <- prop.table(onsite_pharm_counts)*100
onsite_pharm_perc


      No      Yes 
61.29613 38.70387 

# HIV training statistics for overall data

In [32]:
# Create a vector of column names representing the HIV topics columns
hiv_topics <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

## HIV topic with greatest number of 1st rankings

In [33]:
# Calculate the counts of 1st rankings for each HIV topic
hiv1_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
hiv1_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [34]:
# Find the HIV topic with the highest count of 1st rankings
hiv1 <- hiv_topics[which.max(hiv1_counts)]
hiv1

[1] "Q198"

In [35]:
# Calculate the percentage of 1st rankings
hiv1_perc <- (hiv1_counts[which.max(hiv1_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv1_perc

Q198 
30.83475

## HIV topic with greatest number of 2nd rankings

In [36]:
# Calculate the counts of 2nd rankings for each HIV topic
hiv2_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
hiv2_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [37]:
# Find the HIV topic with the highest count of 2nd rankings
hiv2 <- hiv_topics[which.max(hiv2_counts)]
hiv2

[1] "Q199"

In [38]:
# Calculate the percentage of 2nd rankings for the top HIV topic
hiv2_perc <- (hiv2_counts[which.max(hiv2_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv2_perc

Q199 
15.16184

## HIV topic with greatest number of 3rd rankings

In [39]:
# Calculate the counts of 3rd rankings for each HIV topic
hiv3_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
hiv3_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [40]:
# Find the HIV topic with the highest count of 3rd rankings
hiv3 <- hiv_topics[which.max(hiv3_counts)]
hiv3

[1] "Q198"

In [41]:
# Calculate the percentage of 3rd rankings
hiv3_perc <- (hiv3_counts[which.max(hiv3_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv3_perc

Q198 
11.41397

## Top 3 HIV training topics

In [42]:
# Calculate the counts for each LGBTQ topic
hiv_topic_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts
# Sort the topics based on their counts in descending order
hiv_sorted_topics <- sort(hiv_topic_counts, decreasing = TRUE)
# Get the top 3 topics
hiv_top_3_topics <- names(hiv_sorted_topics)[1:3]

hiv_top_3_topics

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  309   219   208    62   121   135   143   156   158    33    68    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

In [43]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents <- sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)

# Calculate the percentage for the first top3_prep_topics
hiv1_percentage <- (hiv_sorted_topics[1] / hiv_total_respondents) * 100

# Calculate the percentage for the second top3_prep_topics
hiv2_percentage <- (hiv_sorted_topics[2] / hiv_total_respondents) * 100

# Calculate the percentage for the third top3_prep_topics
hiv3_percentage <- (hiv_sorted_topics[3] / hiv_total_respondents) * 100

# Print the results
hiv1_percentage
hiv2_percentage
hiv3_percentage

Q198 
52.64055

Q199 
37.30835

Q200 
35.43441

# PrEP training statistics for overall data

In [44]:
# Create a vector of column names representing the PrEP topics columns
prep_topics <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

## PrEP topic with greatest number of 1st rankings

In [45]:
# Calculate the counts of 1st rankings for each PrEP topic
prep1_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
prep1_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [46]:
# Find the PrEP topic with the highest count of 1st rankings
prep1 <- prep_topics[which.max(prep1_counts)]
prep1

[1] "Q211"

In [47]:
# Calculate the percentage of 1st rankings
prep1_perc <- (prep1_counts[which.max(prep1_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep1_perc

Q211 
19.2846

## PrEP topic with greatest number of 2nd rankings

In [48]:
# Calculate the counts of 2nd rankings for each PrEP topic
prep2_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
prep2_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [49]:
# Find the PrEP topic with the highest count of 2nd rankings
prep2 <- prep_topics[which.max(prep2_counts)]
prep2

[1] "Q214"

In [50]:
# Calculate the percentage of 2nd rankings for the top PrEP topic
prep2_perc <- (prep2_counts[which.max(prep2_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep2_perc

Q214 
12.44168

## PrEP topic with greatest number of third rankings

In [51]:
# Calculate the counts of 3rd rankings for each PrEP topic
prep3_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
prep3_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [52]:
# Find the PrEP topic with the highest count of 3rd rankings
prep3 <- prep_topics[which.max(prep3_counts)]
prep3

[1] "Q218"

In [53]:
# Calculate the percentage of 3rd rankings
prep3_perc <- (prep3_counts[which.max(prep3_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep3_perc

Q218 
10.73095

## Top 3 PrEP training topics

In [54]:
# Calculate the counts for each PrEP topic
prep_topic_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
prep_topic_counts
# Sort the topics based on their counts in descending order
prep_sorted_topics <- sort(prep_topic_counts, decreasing = TRUE)

# Get the top 3 topics
prep_top_3_topics <- names(prep_sorted_topics)[1:3]

prep_top_3_topics

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 197  164  158  240   55   91   95  194  113  136   85  102  119   68    8

[1] "Q214" "Q211" "Q218"

In [55]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the PrEP topic questions
prep_total_respondents <- sum(rowSums(!is.na(final_data[, prep_topics])) > 0)

# Calculate the percentage for the first top3_prep_topics
prep1_percentage <- (prep_sorted_topics[1] / prep_total_respondents) * 100

# Calculate the percentage for the second top3_prep_topics
prep2_percentage <- (prep_sorted_topics[2] / prep_total_respondents) * 100

# Calculate the percentage for the third top3_prep_topics
prep3_percentage <- (prep_sorted_topics[3] / prep_total_respondents) * 100

# Print the results
prep1_percentage
prep2_percentage
prep3_percentage

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

# LGBTQ training statistics for overall data

In [56]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

## LGBTQ topic with greatest number of 1st rankings

In [57]:
# Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
lgbtq1_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 139  198  121  120   34   67  118  114   15

In [58]:
# Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1 <- lgbtq_topics[which.max(lgbtq1_counts)]
lgbtq1

[1] "Q228"

In [59]:
# Calculate the percentage of 1st rankings
lgbtq1_perc <- (lgbtq1_counts[which.max(lgbtq1_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq1_perc

Q228 
21.10874

## LGBTQ topic with greatest number of 2nd rankings

In [60]:
# Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
lgbtq2_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  67  121  102  154   62   78  143  143    4

In [61]:
# Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2 <- lgbtq_topics[which.max(lgbtq2_counts)]
lgbtq2

[1] "Q230"

In [62]:
# Calculate the percentage of 2nd rankings for the top LGBTQ topic
lgbtq2_perc <- (lgbtq2_counts[which.max(lgbtq2_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq2_perc

Q230 
16.41791

## LGBTQ topic with greatest number of third rankings

In [63]:
# Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
lgbtq3_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  81  122   73  124   66   64  109  194   14

In [64]:
# Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3 <- lgbtq_topics[which.max(lgbtq3_counts)]
lgbtq3

[1] "Q234"

In [65]:
# Calculate the percentage of 3rd rankings
lgbtq3_perc <- (lgbtq3_counts[which.max(lgbtq3_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq3_perc

Q234 
20.6823

## Top 3 LGBTQ training topics 

In [66]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
lgbtq_topic_counts
# Sort the topics based on their counts in descending order
lgbtq_sorted_topics <- sort(lgbtq_topic_counts, decreasing = TRUE)

# Get the top 3 topics
lgbtq_top_3_topics <- names(lgbtq_sorted_topics)[1:3]

lgbtq_top_3_topics

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 287  441  296  398  162  209  370  451   33

[1] "Q234" "Q228" "Q230"

In [67]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgtbq_total_respondents <- sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)

# Calculate the percentage for the first top3_lgbtq_topics
lgbtq1_percentage <- (lgbtq_sorted_topics[1] / lgtbq_total_respondents) * 100

# Calculate the percentage for the second top3_lgbtq_topics
lgbtq2_percentage <- (lgbtq_sorted_topics[2] / lgtbq_total_respondents) * 100

# Calculate the percentage for the third top3_lgbtq_topics
lgbtq3_percentage <- (lgbtq_sorted_topics[3] / lgtbq_total_respondents) * 100

# Print the results
lgbtq1_percentage
lgbtq2_percentage
lgbtq3_percentage

Q234 
48.08102

Q228 
47.01493

Q230 
42.4307

# Identifying HIV providers

In [68]:
# HIV providers
HIV_providers <- final_data[final_data$Q22 == "Yes" | final_data$Q30 == "Yes" | final_data$Q32 == "Yes", ]
head(HIV_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete
abilly@iuhealth.org,774,AIDS Service Organization (ASO),NA,Indiana,47909,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses among HIV providers

In [69]:
# Complete vs Incomplete among HIV providers
table(HIV_providers$Q819)


  Complete Incomplete 
       760        271 

## Descriptive Statistics among HIV providers

### Age counts among HIV providers

In [70]:
## HIV providers - Age
age_HIV <- table(HIV_providers$Q162)
age_HIV


                18-24                 25-34                 35-44 
                   31                   196                   239 
                45-54                 55-64                    65 
                  249                   204                    91 
Prefer not to respond 
                   19 

### Age percentages among HIV providers

In [71]:
age_perc_HIV <- prop.table(age_HIV)*100
age_perc_HIV


                18-24                 25-34                 35-44 
             3.012634             19.047619             23.226433 
                45-54                 55-64                    65 
            24.198251             19.825073              8.843537 
Prefer not to respond 
             1.846453 

### Gender counts among HIV providers

In [72]:
## HIV providers - Gender
gender_HIV<- table(HIV_providers$Q157)
gender_HIV


                                Female/Woman 
                                         636 
Genderqueer/gender non-confirming/non-binary 
                                          64 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          18 
                           Transgender woman 
                                          14 

### Gender percentages among HIV providers

In [73]:
gender_perc_HIV <- prop.table(gender_HIV)*100
gender_perc_HIV


                                Female/Woman 
                                  61.8075802 
Genderqueer/gender non-confirming/non-binary 
                                   6.2196307 
                                    Male/Man 
                                  26.3362488 
                       Prefer not to respond 
                                   1.6520894 
                     Prefer to self-describe 
                                   0.8746356 
                             Transgender man 
                                   1.7492711 
                           Transgender woman 
                                   1.3605442 

### Race counts among HIV providers

In [74]:
## HIV providers - Race
race_HIV <- table(HIV_providers$Q159)
race_HIV


        Asian and/or Asian American           Black or African American 
                                 26                                 250 
                        Multiracial    Native American or Alaska Native 
                                 77                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  52 
            Prefer to self-describe                  White or Caucasian 
                                 25                                 582 

### Race percentages among HIV providers

In [75]:
race_perc_HIV <- prop.table(race_HIV)*100
race_perc_HIV


        Asian and/or Asian American           Black or African American 
                          2.5291829                          24.3190661 
                        Multiracial    Native American or Alaska Native 
                          7.4902724                           1.0700389 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4863813                           5.0583658 
            Prefer to self-describe                  White or Caucasian 
                          2.4319066                          56.6147860 

### Hispanic/Latinx counts among HIV providers

In [76]:
## HIV providers - Hispanic/Latinx
hisp_HIV <- table(HIV_providers$Q161)
hisp_HIV


                   No Prefer not to respond                   Yes 
                  819                    19                   191 

### Hispanic/Latinx percentages among HIV providers

In [77]:
hisp_perc_HIV<- prop.table(hisp_HIV)*100
hisp_perc_HIV


                   No Prefer not to respond                   Yes 
            79.591837              1.846453             18.561710 

### Geographic counts among HIV providers

In [78]:
geog_counts_HIV <- table(HIV_providers$Q5)
geog_counts_HIV 


   Rural Suburban   Tribal    Urban 
     215      171        3      642 

### Geographic percentages among HIV providers

In [79]:
geog_perc_HIV <- prop.table(geog_counts_HIV)*100
geog_perc_HIV


     Rural   Suburban     Tribal      Urban 
20.8535403 16.5858390  0.2909796 62.2696411 

### Setting counts among HIV providers

In [80]:
setting_counts_HIV <- table(HIV_providers$Q1)
setting_counts_HIV


             Academic hospital/clinic setting 
                                           76 
                        Advocacy organization 
                                            4 
              AIDS Service Organization (ASO) 
                                          132 
           Community Based Organization (CBO) 
                                          111 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                           95 
               Health department (non-clinic) 
                                           39 
Health department clinic/public health clinic 
                                          144 
         LBGTQ Health Center/Community Center 
                                           51 
                      Non-profit organization 
                                          140 
                                        Other 
            

### Setting percentages among HIV providers

In [81]:
setting_perc_HIV <- prop.table(setting_counts_HIV)*100
setting_perc_HIV


             Academic hospital/clinic setting 
                                    7.3714840 
                        Advocacy organization 
                                    0.3879728 
              AIDS Service Organization (ASO) 
                                   12.8031038 
           Community Based Organization (CBO) 
                                   10.7662464 
     Federally Qualified Health Center (FQHC) 
                                   12.9970902 
    Government entity (Local, State, Federal) 
                                    9.2143550 
               Health department (non-clinic) 
                                    3.7827352 
Health department clinic/public health clinic 
                                   13.9670223 
         LBGTQ Health Center/Community Center 
                                    4.9466537 
                      Non-profit organization 
                                   13.5790495 
                                        Other 
            

### Role type counts among HIV providers - clinical vs non-clinical

In [82]:
role_type_counts_HIV <- table(HIV_providers$Q65)
role_type_counts_HIV


    Clinical Non-clinical 
         344          686 

### Role type percentages among HIV providers - clinical vs non-clinical

In [83]:
role_type_perc_HIV <- prop.table(role_type_counts_HIV)*100
role_type_perc_HIV


    Clinical Non-clinical 
    33.39806     66.60194 

### Clinical role counts among HIV providers

In [84]:
role_counts_HIV <- table(HIV_providers$Q66)
role_counts_HIV


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  37                                   10 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   12 
                       Nurse Manager                        Nurse Midwife 
                                  18                                    1 
                  Nurse Practitioner Other Provider/Clinical professional 
                                  68                                    7 
                          Pharmacist                  Pharmacy Technician 
                        

### Clinical role percentages among HIV providers

In [85]:
role_perc_HIV <- prop.table(role_counts_HIV)*100
role_perc_HIV


                             Dentist      Disease Intervention Specialist 
                           0.5830904                            0.2915452 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          10.7871720                            2.9154519 
                   Medical Assistant           Mental Health Case Manager 
                           2.9154519                            0.8746356 
             Mental Health Counselor           Mental Health Professional 
                           0.2915452                            3.4985423 
                       Nurse Manager                        Nurse Midwife 
                           5.2478134                            0.2915452 
                  Nurse Practitioner Other Provider/Clinical professional 
                          19.8250729                            2.0408163 
                          Pharmacist                  Pharmacy Technician 
                        

### Non-clinical role counts among HIV providers

In [86]:
nonclinic_role_counts_HIV <- table(HIV_providers$Q71)
nonclinic_role_counts_HIV


                         Administrator                               Advocate 
                                   157                                     15 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                                   123                                      1 
               Community Health Worker        Disease Intervention Specialist 
                                    64                                     37 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                                     2                                     32 
             Health and Wellness Coach            Health Education Specialist 
                                     1                                     56 
                      Health Navigator                HIV Linkage Coordinator 
                                    35                                      4 
             HIV Retention Coordinator             

### Non-clinical role percentages among HIV providers

In [87]:
nonclinic_role_perc_HIV <- prop.table(nonclinic_role_counts_HIV)*100
nonclinic_role_perc_HIV


                         Administrator                               Advocate 
                            22.9532164                              2.1929825 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                            17.9824561                              0.1461988 
               Community Health Worker        Disease Intervention Specialist 
                             9.3567251                              5.4093567 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                             0.2923977                              4.6783626 
             Health and Wellness Coach            Health Education Specialist 
                             0.1461988                              8.1871345 
                      Health Navigator                HIV Linkage Coordinator 
                             5.1169591                              0.5847953 
             HIV Retention Coordinator             

### Years in role counts among HIV providers

In [88]:
years_counts_HIV <- table(HIV_providers$Q94)
years_counts_HIV


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        285         150         219         157         219 

### Years in role percentages among HIV providers

In [89]:
years_perc_HIV <- prop.table(years_counts_HIV)*100
years_perc_HIV


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   27.66990    14.56311    21.26214    15.24272    21.26214 

### Prescriber counts among HIV providers

In [90]:
prescriber_counts_HIV <- table(HIV_providers$Q70)
prescriber_counts_HIV


 No Yes 
198 149 

### Prescriber percentages among HIV providers

In [91]:
prescriber_perc_HIV <- prop.table(prescriber_counts_HIV)*100
prescriber_perc_HIV


      No      Yes 
57.06052 42.93948 

### Onsite pharmacy counts among HIV providers

In [92]:
onsite_pharm_counts_HIV <- table(HIV_providers$Q6)
onsite_pharm_counts_HIV


 No Yes 
613 418 

### Onsite pharmacy percentages among HIV providers

In [93]:
onsite_pharm_perc_HIV <- prop.table(onsite_pharm_counts_HIV)*100
onsite_pharm_perc_HIV


      No      Yes 
59.45684 40.54316 

## HIV training statistics for HIV providers

In [94]:
# HIV providers - Create a vector of column names representing the HIV topics columns
hiv_topics_HIV <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among HIV providers

In [95]:
# HIV providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [96]:
# HIV providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_HIV <- hiv_topics_HIV[which.max(hiv1_count_HIV)]
hiv1_rank_HIV

[1] "Q198"

In [97]:
# HIV providers - Calculate the percentage of 1st rankings
hiv1_perc_HIV <- (hiv1_count_HIV[which.max(hiv1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv1_perc_HIV

Q198 
30.83475

### HIV topic with greatest number of 2nd rankings among HIV providers

In [98]:
# HIV providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [99]:
# HIV providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_HIV <- hiv_topics_HIV[which.max(hiv2_count_HIV)]
hiv2_rank_HIV

[1] "Q199"

In [100]:
# HIV providers - Calculate the percentage of 2nd ranking
hiv2_perc_HIV <- (hiv2_count_HIV[which.max(hiv2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv2_perc_HIV

Q199 
15.16184

### HIV topic with greatest number of 3rd rankings among HIV providers

In [101]:
# HIV providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [102]:
# HIV providers - Find the HIV topic with the highest count of 3rd rankings
hiv3_rank_HIV <- hiv_topics_HIV[which.max(hiv3_count_HIV)]
hiv3_rank_HIV

[1] "Q198"

In [103]:
# HIV providers - Calculate the percentage of 3rd rankings
hiv3_perc_HIV <- (hiv3_count_HIV[which.max(hiv3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv3_perc_HIV

Q198 
11.41397

### Top 3 HIV topics among HIV providers

In [104]:
# Calculate the counts for each HIV topic
hiv_topic_counts_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
hiv_sorted_topics_HIV <- sort(hiv_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_hiv_topics_HIV <- names(hiv_sorted_topics_HIV)[1:3]

top3_hiv_topics_HIV

[1] "Q198" "Q199" "Q200"

### Percentage of each top 3 HIV topic among HIV providers

In [105]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
hiv1_percentage_HIV <- (hiv_sorted_topics_HIV[1] / hiv_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
hiv2_percentage_HIV <- (hiv_sorted_topics_HIV[2] / hiv_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
hiv3_percentage_HIV <- (hiv_sorted_topics_HIV[3] / hiv_total_respondents_HIV) * 100

# Print the results
hiv1_percentage_HIV
hiv2_percentage_HIV
hiv3_percentage_HIV

Q198 
52.64055

Q199 
37.30835

Q200 
35.43441

## Prep training statistics for HIV providers

In [106]:
# HIV providers - Create a vector of column names representing the PrEP topics columns
prep_topics_HIV <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among HIV providers

In [107]:
# HIV providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [108]:
# HIV providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_HIV <- prep_topics_HIV[which.max(prep1_count_HIV)]
prep1_rank_HIV

[1] "Q211"

In [109]:
# HIV providers - Calculate the percentage of 1st rankings
prep1_perc_HIV <- (prep1_count_HIV[which.max(prep1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep1_perc_HIV

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among HIV providers

In [110]:
# HIV providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [111]:
# HIV providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_HIV <- prep_topics_HIV[which.max(prep2_count_HIV)]
prep2_rank_HIV

[1] "Q214"

In [112]:
# HIV providers - Calculate the percentage of 2nd ranking
prep2_perc_HIV <- (prep2_count_HIV[which.max(prep2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep2_perc_HIV

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among HIV providers

In [113]:
# HIV providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [114]:
# HIV providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_HIV <- prep_topics_HIV[which.max(prep3_count_HIV)]
prep3_rank_HIV

[1] "Q218"

In [115]:
# HIV providers - Calculate the percentage of 3rd rankings
prep3_perc_HIV <- (prep3_count_HIV[which.max(prep3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep3_perc_HIV

Q218 
10.73095

### Top 3 PrEP topics among HIV providers

In [116]:
# Calculate the counts for each PrEP topic
prep_topic_counts_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_HIV <- sort(prep_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_HIV <- names(prep_sorted_topics_HIV)[1:3]

top3_prep_topics_HIV

[1] "Q214" "Q211" "Q218"

### Percentage of each top 3 PrEP topic among HIV providers

In [117]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
prep_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
prep1_percentage_HIV <- (prep_sorted_topics_HIV[1] / prep_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
prep2_percentage_HIV <- (prep_sorted_topics_HIV[2] / prep_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
prep3_percentage_HIV <- (prep_sorted_topics_HIV[3] / prep_total_respondents_HIV) * 100

# Print the results
prep1_percentage_HIV
prep2_percentage_HIV
prep3_percentage_HIV

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

## LGBTQ training statistcis for HIV providers

In [118]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_HIV <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among HIV providers

In [119]:
# HIV providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  186  113  110   29   63  112  109   11

In [120]:
# HIV providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq1_count_HIV)]
lgbtq1_rank_HIV

[1] "Q228"

In [121]:
# HIV providers - Calculate the percentage of 1st rankings
lgbtq1_perc_HIV <- (lgbtq1_count_HIV[which.max(lgbtq1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq1_perc_HIV

Q228 
21.20867

### LGBTQ topic with greatest number of 2nd rankings among HIV providers

In [122]:
# HIV providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  63  114   99  142   58   73  128  134    3

In [123]:
# HIV providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq2_count_HIV)]
lgbtq2_rank_HIV

[1] "Q230"

In [124]:
# HIV providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_HIV <- (lgbtq2_count_HIV[which.max(lgbtq2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq2_perc_HIV

Q230 
16.19156

### LGBTQ topic with greatest number of 3rd rankings among HIV providers

In [125]:
# HIV providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
lgbtq3_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  74  115   69  115   63   60  104  177   13

In [126]:
# HIV providers - Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq3_count_HIV)]
lgbtq3_rank_HIV

[1] "Q234"

In [127]:
# HIV providers - Calculate the percentage of 3rd rankings
lgbtq3_perc_HIV <- (lgbtq3_count_HIV[which.max(lgbtq3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq3_perc_HIV

Q234 
20.18244

### Top 3 LGBTQ topics among HIV providers

In [128]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_HIV <- sort(lgbtq_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_HIV <- names(lgbtq_sorted_topics_HIV)[1:3]
top3_lgbtq_topics_HIV

[1] "Q234" "Q228" "Q230"

### Percentage of each top 3 LGBTQ topic among HIV providers

In [129]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
lgbtq_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
lgbtq1_percentage_HIV <- (lgbtq_sorted_topics_HIV[1] / lgbtq_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
lgbtq2_percentage_HIV <- (lgbtq_sorted_topics_HIV[2] / lgbtq_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
lgbtq3_percentage_HIV <- (lgbtq_sorted_topics_HIV[3] / lgbtq_total_respondents_HIV) * 100

# Print the results
lgbtq1_percentage_HIV
lgbtq2_percentage_HIV
lgbtq3_percentage_HIV

Q234 
47.89054

Q228 
47.32041

Q230 
41.84721

## Identifying LGBTQ providers

In [130]:
# LGBTQ providers
LGBTQ_providers <- final_data[final_data$Q275 == "Yes" | final_data$Q130 == 1 | final_data$Q1 == "LBGTQ Health Center/Community Center" | final_data$Q22 == "Yes" | final_data$Q33 == 1, ]
head(LGBTQ_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses among LGBTQ providers

In [131]:
table(LGBTQ_providers$Q819)


  Complete Incomplete 
       754        269 

## Descriptive Statistics for LGBTQ providers

### Age counts among LGBTQ providers

In [132]:
## LGBTQ providers - Age
age_LGBTQ <- table(LGBTQ_providers$Q162)
age_LGBTQ


                18-24                 25-34                 35-44 
                   31                   196                   241 
                45-54                 55-64                    65 
                  243                   202                    88 
Prefer not to respond 
                   19 

### Age percentages among LGBTQ providers

In [133]:
age_perc_LGBTQ <- prop.table(age_LGBTQ)*100
age_perc_LGBTQ


                18-24                 25-34                 35-44 
             3.039216             19.215686             23.627451 
                45-54                 55-64                    65 
            23.823529             19.803922              8.627451 
Prefer not to respond 
             1.862745 

### Gender counts among LGBTQ providers

In [134]:
## LGBTQ providers - Gender
gender_LGBTQ <- table(LGBTQ_providers$Q157)
gender_LGBTQ


                                Female/Woman 
                                         625 
Genderqueer/gender non-confirming/non-binary 
                                          65 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          19 
                           Transgender woman 
                                          14 

### Gender percentages among LGBTQ providers

In [135]:
gender_perc_LGBTQ <- prop.table(gender_LGBTQ)*100
gender_perc_LGBTQ


                                Female/Woman 
                                  61.2745098 
Genderqueer/gender non-confirming/non-binary 
                                   6.3725490 
                                    Male/Man 
                                  26.5686275 
                       Prefer not to respond 
                                   1.6666667 
                     Prefer to self-describe 
                                   0.8823529 
                             Transgender man 
                                   1.8627451 
                           Transgender woman 
                                   1.3725490 

### Race counts among LGBTQ providers

In [136]:
## LGBTQ providers - Race
race_LGBTQ <- table(LGBTQ_providers$Q159)
race_LGBTQ


        Asian and/or Asian American           Black or African American 
                                 25                                 251 
                        Multiracial    Native American or Alaska Native 
                                 73                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  53 
            Prefer to self-describe                  White or Caucasian 
                                 24                                 577 

### Race percentages among LGBTQ providers

In [137]:
race_perc_LGBTQ <- prop.table(race_LGBTQ)*100
race_perc_LGBTQ


        Asian and/or Asian American           Black or African American 
                          2.4533857                          24.6319921 
                        Multiracial    Native American or Alaska Native 
                          7.1638862                           1.0794897 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4906771                           5.2011776 
            Prefer to self-describe                  White or Caucasian 
                          2.3552502                          56.6241413 

### Hispanic/Latinx percentages among LGBTQ providers

In [138]:
## LGBTQ providers - Hispanic/Latinx
hisp_LGBTQ <- table(LGBTQ_providers$Q161)
hisp_LGBTQ


                   No Prefer not to respond                   Yes 
                  814                    19                   187 

### Hispanic/Latinx percentages among LGBTQ providers

In [139]:
hisp_perc_LGBTQ <- prop.table(hisp_LGBTQ)*100
hisp_perc_LGBTQ 


                   No Prefer not to respond                   Yes 
            79.803922              1.862745             18.333333 

### Geographic counts among LGBTQ providers

In [140]:
geog_counts_LGBTQ <- table(LGBTQ_providers$Q5)
geog_counts_LGBTQ


   Rural Suburban   Tribal    Urban 
     206      175        3      639 

### Geographic percentages among LGBTQ providers

In [141]:
geog_perc_LGBTQ <- prop.table(geog_counts_LGBTQ)*100
geog_perc_LGBTQ


     Rural   Suburban     Tribal      Urban 
20.1368524 17.1065494  0.2932551 62.4633431 

### Setting counts among LGBTQ providers

In [142]:
setting_counts_LGBTQ <- table(LGBTQ_providers$Q1)
setting_counts_LGBTQ


             Academic hospital/clinic setting 
                                           74 
                        Advocacy organization 
                                            4 
              AIDS Service Organization (ASO) 
                                          133 
           Community Based Organization (CBO) 
                                          116 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                           88 
               Health department (non-clinic) 
                                           38 
Health department clinic/public health clinic 
                                          137 
         LBGTQ Health Center/Community Center 
                                           55 
                      Non-profit organization 
                                          138 
                                        Other 
            

### Setting percentages among LGBTQ providers

In [143]:
setting_perc_LGBTQ <- prop.table(setting_counts_LGBTQ)*100
setting_perc_LGBTQ


             Academic hospital/clinic setting 
                                    7.2336266 
                        Advocacy organization 
                                    0.3910068 
              AIDS Service Organization (ASO) 
                                   13.0009775 
           Community Based Organization (CBO) 
                                   11.3391984 
     Federally Qualified Health Center (FQHC) 
                                   13.0987292 
    Government entity (Local, State, Federal) 
                                    8.6021505 
               Health department (non-clinic) 
                                    3.7145650 
Health department clinic/public health clinic 
                                   13.3919844 
         LBGTQ Health Center/Community Center 
                                    5.3763441 
                      Non-profit organization 
                                   13.4897361 
                                        Other 
            

### Role type counts among LGBTQ providers - clinical vs nonclinical

In [144]:
role_type_counts_LGBTQ <- table(LGBTQ_providers$Q65)
role_type_counts_LGBTQ


    Clinical Non-clinical 
         336          685 

### Role type percentages among LGBTQ providers - clinical vs nonclinical

In [145]:
role_type_perc_LGBTQ <- prop.table(role_type_counts_LGBTQ)*100
role_type_perc_LGBTQ


    Clinical Non-clinical 
    32.90891     67.09109 

### Clinical role counts among LGBTQ providers

In [146]:
role_counts_LGBTQ <- table(LGBTQ_providers$Q66)
role_counts_LGBTQ


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  39                                    7 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   12 
                       Nurse Manager                        Nurse Midwife 
                                  17                                    1 
                  Nurse Practitioner Other Provider/Clinical professional 
                                  68                                    7 
                          Pharmacist                  Pharmacy Technician 
                        

### Clinical role percentages among LGBTQ providers

In [147]:
role_perc_LGBTQ <- prop.table(role_counts_LGBTQ)*100
role_perc_LGBTQ


                             Dentist      Disease Intervention Specialist 
                           0.5970149                            0.2985075 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          11.6417910                            2.0895522 
                   Medical Assistant           Mental Health Case Manager 
                           2.9850746                            0.8955224 
             Mental Health Counselor           Mental Health Professional 
                           0.2985075                            3.5820896 
                       Nurse Manager                        Nurse Midwife 
                           5.0746269                            0.2985075 
                  Nurse Practitioner Other Provider/Clinical professional 
                          20.2985075                            2.0895522 
                          Pharmacist                  Pharmacy Technician 
                        

### Non-clinical role counts among LGBTQ providers

In [148]:
nonclinic_role_counts_LGBTQ <- table(LGBTQ_providers$Q71)
nonclinic_role_counts_LGBTQ


                         Administrator                               Advocate 
                                   160                                     15 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                                   122                                      1 
               Community Health Worker        Disease Intervention Specialist 
                                    63                                     36 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                                     2                                     29 
             Health and Wellness Coach            Health Education Specialist 
                                     1                                     55 
                      Health Navigator                HIV Linkage Coordinator 
                                    35                                      4 
             HIV Retention Coordinator             

### Non-clinical role percentages among LGBTQ providers

In [149]:
nonclinic_role_perc_HIV <- prop.table(nonclinic_role_counts_HIV)*100
nonclinic_role_perc_HIV


                         Administrator                               Advocate 
                            22.9532164                              2.1929825 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                            17.9824561                              0.1461988 
               Community Health Worker        Disease Intervention Specialist 
                             9.3567251                              5.4093567 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                             0.2923977                              4.6783626 
             Health and Wellness Coach            Health Education Specialist 
                             0.1461988                              8.1871345 
                      Health Navigator                HIV Linkage Coordinator 
                             5.1169591                              0.5847953 
             HIV Retention Coordinator             

### Years in role counts among LGBTQ providers

In [150]:
years_counts_LGBTQ <- table(LGBTQ_providers$Q94)
years_counts_LGBTQ


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        284         146         216         153         222 

### Years in role percentages among LGBTQ providers

In [151]:
years_perc_LGBTQ <- prop.table(years_counts_LGBTQ)*100
years_perc_LGBTQ


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   27.81587    14.29971    21.15573    14.98531    21.74339 

### Prescriber counts among LGBTQ providers

In [152]:
prescriber_counts_LGBTQ <- table(LGBTQ_providers$Q70)
prescriber_counts_LGBTQ


 No Yes 
193 146 

### Prescriber percentages among LGBTQ providers

In [153]:
prescriber_perc_LGBTQ <- prop.table(prescriber_counts_LGBTQ)*100
prescriber_perc_LGBTQ


      No      Yes 
56.93215 43.06785 

## Onsite pharmacy counts among LGBTQ providers

In [154]:
onsite_pharm_counts_LGBTQ <- table(LGBTQ_providers$Q6)
onsite_pharm_counts_LGBTQ


 No Yes 
613 410 

## Onsite pharmacy percentages among LGBTQ providers

In [155]:
onsite_pharm_perc_LGBTQ <- prop.table(onsite_pharm_counts_LGBTQ)*100
onsite_pharm_perc_LGBTQ


     No     Yes 
59.9218 40.0782 

## HIV training statistcis for LGBTQ providers

In [156]:
# LGBTQ providers - Create a vector of column names representing the HIV topics columns
hiv_topics_LGBTQ <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among LGBTQ providers

In [157]:
# LGBTQ providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  178    80    72    17    21    37    34    49    48     6    12    12     3 
Q210b 
    1

In [158]:
# LGBTQ providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv1_count_LGBTQ)]
hiv1_rank_LGBTQ

[1] "Q198"

In [159]:
# LGBTQ providers - Calculate the percentage of 1st rankings
hiv1_perc_LGBTQ <- (hiv1_count_LGBTQ[which.max(hiv1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv1_perc_LGBTQ

Q198 
30.95652

### HIV topic with greatest number of 2nd rankings among LGBTQ providers

In [160]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   56    87    76    26    42    51    48    49    56    15    19     2     0 
Q210b 
    1

In [161]:
# LGBTQ providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv2_count_LGBTQ)]
hiv2_rank_LGBTQ

[1] "Q199"

In [162]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
hiv2_perc_LGBTQ  <- (hiv2_count_LGBTQ[which.max(hiv2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv2_perc_LGBTQ 

Q199 
15.13043

### HIV topic with greatest number of 3rd rankings among LGBTQ providers

In [163]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_LGBTQ  <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   65    47    53    17    56    46    57    58    52    12    36    16     2 
Q210b 
    3

In [164]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
hiv3_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv3_count_LGBTQ)]
hiv3_rank_LGBTQ

[1] "Q198"

In [165]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
hiv3_perc_LGBTQ <- (hiv3_count_LGBTQ[which.max(hiv3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv3_perc_LGBTQ

Q198 
11.30435

### Top 3 HIV topics among LGBTQ providers

In [166]:
# Calculate the counts for each HIV topic
hiv_topic_counts_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts_LGBTQ
# Sort the topics based on their counts in descending order
hiv_sorted_topics_LGBTQ <- sort(hiv_topic_counts_LGBTQ, decreasing = TRUE)
hiv_sorted_topics_LGBTQ
# Get the top 3 topics
top3_hiv_topics_LGBTQ <- names(hiv_sorted_topics_LGBTQ)[1:3]

top3_hiv_topics_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  299   214   201    60   119   134   139   156   156    33    67    30     5 
Q210b 
    5

Q198  Q199  Q200  Q205  Q206  Q204  Q203  Q202  Q208  Q201  Q207  Q209 Q210a 
  299   214   201   156   156   139   134   119    67    60    33    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

### Percentage of each top 3 HIV topic among LGBTQ providers

In [167]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_hiv_topics_LGBTQ
hiv1_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[1] / hiv_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_hiv_topics_LGBTQ
hiv2_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[2] / hiv_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_hiv_topics_LGBTQ
hiv3_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[3] / hiv_total_respondents_LGBTQ) * 100

# Print the results
hiv1_percentage_LGBTQ
hiv2_percentage_LGBTQ
hiv3_percentage_LGBTQ

Q198 
  52

Q199 
37.21739

Q200 
34.95652

## PrEP training statistcis for LGBTQ providers

In [168]:
# LGBTQ providers - Create a vector of column names representing the PrEP topics columns
prep_topics_LGBTQ <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among LGBTQ providers

In [169]:
# LGBTQ providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [170]:
# LGBTQ providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep1_count_LGBTQ)]
prep1_rank_LGBTQ

[1] "Q211"

In [171]:
# LGBTQ providers - Calculate the percentage of 1st rankings
prep1_perc_LGBTQ <- (prep1_count_LGBTQ[which.max(prep1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep1_perc_LGBTQ

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among LGBTQ providers

In [172]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [173]:
# LGBTQ providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep2_count_LGBTQ)]
prep2_rank_LGBTQ

[1] "Q214"

In [174]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
prep2_perc_LGBTQ  <- (prep2_count_LGBTQ[which.max(prep2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep2_perc_LGBTQ 

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among LGBTQ providers

In [175]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_LGBTQ  <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [176]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep3_count_LGBTQ )]
prep3_rank_LGBTQ

[1] "Q218"

In [177]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
prep3_perc_LGBTQ <- (prep3_count_LGBTQ[which.max(prep3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep3_perc_LGBTQ

Q218 
10.73095

### Top 3 PrEP topics among LGBTQ providers

In [178]:
# Calculate the counts for each PrEP topic
prep_topic_counts_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_LGBTQ <- sort(prep_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_LGBTQ <- names(prep_sorted_topics_LGBTQ)[1:3]
top3_prep_topics_LGBTQ

[1] "Q214" "Q211" "Q218"

### Percentage of each top 3 PrEP topic among LGBTQ providers

In [179]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the PrEP topic questions
prep_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_prep_topics_LGBTQ
prep1_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[1] / prep_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_prep_topics_LGBTQ
prep2_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[2] / prep_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_prep_topics_LGBTQ
prep3_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[3] / prep_total_respondents_LGBTQ) * 100

# Print the results
prep1_percentage_LGBTQ
prep2_percentage_LGBTQ
prep3_percentage_LGBTQ

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

## LGBTQ training statistcis for LGBTQ providers

In [180]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_LGBTQ <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among LGBTQ providers

In [181]:
# LGBTQ providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  178  114  111   29   58  110  108   14

In [182]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq1_count_LGBTQ)]
lgbtq1_rank_LGBTQ

[1] "Q228"

In [183]:
# LGBTQ providers - Calculate the percentage of 1st rankings
lgbtq1_perc_LGBTQ <- (lgbtq1_count_LGBTQ[which.max(lgbtq1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq1_perc_LGBTQ

Q228 
20.57803

### LGBTQ topic with greatest number of 2nd rankings among LGBTQ providers

In [184]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [185]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq2_count_LGBTQ)]
lgbtq2_rank_LGBTQ

[1] "Q230"

In [186]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_LGBTQ <- (lgbtq2_count_LGBTQ[which.max(lgbtq2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq2_perc_LGBTQ

Q230 
15.83815

### LGBTQ topic with greatest number of 3rd rankings among LGBTQ providers

In [187]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq3_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [188]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq3_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq3_count_LGBTQ)]
lgbtq3_rank_LGBTQ

[1] "Q230"

In [189]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq3_perc_LGBTQ <- (lgbtq3_count_LGBTQ[which.max(lgbtq3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq3_perc_LGBTQ

Q230 
15.83815

### Top 3 LGBTQ topics among LGBTQ providers

In [190]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_LGBTQ <- sort(lgbtq_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_LGBTQ <- names(lgbtq_sorted_topics_LGBTQ)[1:3]
top3_lgbtq_topics_LGBTQ

[1] "Q234" "Q228" "Q230"

### Percentage of each top 3 LGBTQ topic among LGBTQ providers


In [191]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgtbq_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_lgbtq_topics_LGBTQ
lgbtq1_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[1] / lgtbq_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_lgbtq_topics_LGBTQ
lgbtq2_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[2] / lgtbq_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_lgbtq_topics_LGBTQ
lgbtq3_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[3] / lgtbq_total_respondents_LGBTQ) * 100

# Print the results
lgbtq1_percentage_LGBTQ
lgbtq2_percentage_LGBTQ
lgbtq3_percentage_LGBTQ

Q234 
47.74566

Q228 
46.47399

Q230 
42.31214

# Insurance data

In [192]:
# Define the insurance types and their corresponding column names
insurance_counts <- final_data %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE)
  )

insurance_counts

Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option
<int>,<int>,<int>,<int>,<int>,<int>,<int>
825,731,663,654,682,606,297


In [193]:
# Calculate the total count across all rows for each insurance option
insurance_columns <- final_data[, c("Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13")]

# Calculate the total count across all types of insurance
total_all_insurance <- sum(insurance_columns, na.rm = TRUE)
total_all_insurance

[1] 4458

In [194]:
# Calculate the percentage of each option
insurance_percentage <- insurance_counts / total_all_insurance * 100

# Print the percentages
insurance_percentage

Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
18.50606,16.39749,14.87214,14.67026,15.29834,13.59354,6.66218


## Insurance data by state

In [195]:
# Calculate the count of each insurance option by state
ins_counts_by_state <- final_data %>%
  group_by(Q3) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by state
ins_counts_by_state

Q3,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Alabama,6,5,5,4,6,4,1,31
Alaska,0,0,0,0,0,0,1,1
Arizona,13,10,7,7,10,9,5,61
Arkansas,7,7,7,6,6,5,2,40
California,73,73,53,48,51,49,32,379
Colorado,12,10,10,9,10,9,5,65
Connecticut,12,12,10,8,7,10,6,65
Delaware,11,11,8,10,10,6,2,58
District of Columbia,26,20,17,14,18,11,9,115


In [196]:
# Calculate the total count across all types of insurance for each state
total_ins_counts_by_state <- final_data %>%
  group_by(Q3) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each state
total_ins_counts_by_state

Q3,Total_Count
<chr>,<dbl>
Alabama,31
Alaska,1
Arizona,61
Arkansas,40
California,379
Colorado,65
Connecticut,65
Delaware,58
District of Columbia,115


In [197]:
# Calculate the percentage of each option by state
ins_perc_by_state <- ins_counts_by_state %>%
    group_by(Q3) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
)
# Print the percentage of each option by state
ins_perc_by_state

Q3,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Alabama,19.35484,16.12903,16.12903,12.903226,19.354839,12.903226,3.225806
Alaska,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,100.000000
Arizona,21.31148,16.39344,11.47541,11.475410,16.393443,14.754098,8.196721
Arkansas,17.50000,17.50000,17.50000,15.000000,15.000000,12.500000,5.000000
California,19.26121,19.26121,13.98417,12.664908,13.456464,12.928760,8.443272
Colorado,18.46154,15.38462,15.38462,13.846154,15.384615,13.846154,7.692308
Connecticut,18.46154,18.46154,15.38462,12.307692,10.769231,15.384615,9.230769
Delaware,18.96552,18.96552,13.79310,17.241379,17.241379,10.344828,3.448276
District of Columbia,22.60870,17.39130,14.78261,12.173913,15.652174,9.565217,7.826087


## Insurance data by CDC region

In [198]:
# Create a new column "Region" based on the states
final_data <- final_data %>%
  mutate(Region = case_when(
    Q3 %in% c("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey", "New York", "Pennsylvania", "Rhode Island", "Vermont") ~ "Northeast",
    Q3 %in% c("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota", "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin") ~ "Midwest",
    Q3 %in% c("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia", "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma", "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia") ~ "South",
    Q3 %in% c("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana", "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming") ~ "West",
    Q3 %in% c("Puerto Rico", "Virgin Islands") ~ "Territories",
    TRUE ~ "Unknown" # If the state is NA
  ))

In [199]:
# Filter rows where the Region is "Unknown"
unknown_states <- filter(final_data, Region == "Unknown")

# Print the states categorized as "Unknown"
print(unknown_states$Q3)

[1] NA NA NA NA


In [200]:
# Calculate the count of each insurance option by region
ins_counts_by_region <- final_data %>%
  group_by(Region) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by state and region
ins_counts_by_region

Region,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Midwest,174,151,137,152,154,133,44,945
Northeast,176,155,146,141,144,119,65,946
South,329,287,264,253,268,252,113,1766
Territories,7,8,7,7,6,4,2,41
Unknown,1,1,1,1,1,1,1,7
West,138,129,108,100,109,97,72,753


In [201]:
# Calculate the total count across all types of insurance for each region
total_ins_counts_by_region <- final_data %>%
  group_by(Region) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each region
total_ins_counts_by_region

Region,Total_Count
<chr>,<dbl>
Midwest,945
Northeast,946
South,1766
Territories,41
Unknown,7
West,753


In [202]:
# Calculate the percentage of each option by region
ins_perc_by_region <- ins_counts_by_region %>%
    group_by(Region) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
  )
# Print the percentage of each option by state and region
ins_perc_by_region

Region,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Midwest,18.41270,15.97884,14.49735,16.08466,16.29630,14.074074,4.656085
Northeast,18.60465,16.38478,15.43340,14.90486,15.22199,12.579281,6.871036
South,18.62967,16.25142,14.94904,14.32616,15.17554,14.269536,6.398641
Territories,17.07317,19.51220,17.07317,17.07317,14.63415,9.756098,4.878049
Unknown,14.28571,14.28571,14.28571,14.28571,14.28571,14.285714,14.285714
West,18.32669,17.13147,14.34263,13.28021,14.47543,12.881806,9.561753


## Insurance data by Medicaid Expansion states

In [203]:
# Create a new column "Medicaid_Expansion_Status" based on Medicaid expansion status
final_data <- final_data  %>%
  mutate(Medicaid_Expansion = case_when(
      Q3 %in% c("Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Hawaii", "Illinois", "Idaho", "Indiana", "Iowa", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "Utah", "Vermont", "Virginia", "Washington", "West Virginia") ~ "Expanded",
      Q3 %in% c("Wyoming", "Kansas", "Texas", "Wisconsin", "Tennessee", "Mississippi", "Alabama", "Georgia", "South Carolina", "Florida") ~ "Not Expanded",
      Q3 %in% c("Puerto Rico", "Virgin Islands") ~ "Territories",
      TRUE ~ "Unknown" # For NA states
  ))

In [204]:
# Filter rows where the Region is "Unknown"
unknown_states <- filter(final_data, Medicaid_Expansion == "Unknown")

# Print the states categorized as "Unknown"
print(unknown_states$Q3)

[1] NA NA NA NA


In [205]:
# Calculate the count of each insurance option by expansion status
ins_counts_by_expansion_status<- final_data %>%
  group_by(Medicaid_Expansion) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by expansion status
ins_counts_by_expansion_status

Medicaid_Expansion,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Expanded,665,589,531,529,541,474,232,3561
Not Expanded,152,133,124,117,134,127,62,849
Territories,7,8,7,7,6,4,2,41
Unknown,1,1,1,1,1,1,1,7


In [206]:
# Calculate the total count across all types of insurance for each expansion status
total_ins_counts_by_expansion_status <- final_data %>%
  group_by(Medicaid_Expansion) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each expansion status
total_ins_counts_by_expansion_status

Medicaid_Expansion,Total_Count
<chr>,<dbl>
Expanded,3561
Not Expanded,849
Territories,41
Unknown,7


In [207]:
# Calculate the percentage of each option by expansion status
ins_perc_by_expansion_status <- ins_counts_by_expansion_status %>%
    group_by(Medicaid_Expansion) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
  )
# Print the percentage of each option by state and region
ins_perc_by_expansion_status

Medicaid_Expansion,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Expanded,18.67453,16.54030,14.91154,14.85538,15.19236,13.310868,6.515024
Not Expanded,17.90342,15.66549,14.60542,13.78092,15.78327,14.958775,7.302709
Territories,17.07317,19.51220,17.07317,17.07317,14.63415,9.756098,4.878049
Unknown,14.28571,14.28571,14.28571,14.28571,14.28571,14.285714,14.285714


## Insurance data by site

In [208]:
# Calculate the count of each insurance option by site
ins_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by site
ins_counts_by_site

Q1,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Academic hospital/clinic setting,76,68,66,71,74,45,8,408
Advocacy organization,1,0,1,1,1,0,3,7
AIDS Service Organization (ASO),102,96,82,84,87,69,40,560
Community Based Organization (CBO),77,64,52,54,56,51,52,406
Federally Qualified Health Center (FQHC),133,130,117,109,124,122,19,754
"Government entity (Local, State, Federal)",66,57,54,50,47,48,29,351
Health department (non-clinic),24,20,20,16,16,15,16,127
Health department clinic/public health clinic,120,106,93,83,86,101,23,612
LBGTQ Health Center/Community Center,32,27,25,22,25,21,16,168


In [209]:
# Calculate the total count across all types of insurance for each site
total_ins_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each site
total_ins_counts_by_site

Q1,Total_Count
<chr>,<dbl>
Academic hospital/clinic setting,408
Advocacy organization,7
AIDS Service Organization (ASO),560
Community Based Organization (CBO),406
Federally Qualified Health Center (FQHC),754
"Government entity (Local, State, Federal)",351
Health department (non-clinic),127
Health department clinic/public health clinic,612
LBGTQ Health Center/Community Center,168


In [210]:
# Calculate the percentage of each option by site
ins_perc_by_site <- ins_counts_by_site %>%
    group_by(Q1) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
)
# Print the percentage of each option by site
ins_perc_by_site

Q1,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Academic hospital/clinic setting,18.62745,16.66667,16.17647,17.40196,18.13725,11.02941,1.960784
Advocacy organization,14.28571,0.00000,14.28571,14.28571,14.28571,0.00000,42.857143
AIDS Service Organization (ASO),18.21429,17.14286,14.64286,15.00000,15.53571,12.32143,7.142857
Community Based Organization (CBO),18.96552,15.76355,12.80788,13.30049,13.79310,12.56158,12.807882
Federally Qualified Health Center (FQHC),17.63926,17.24138,15.51724,14.45623,16.44562,16.18037,2.519894
"Government entity (Local, State, Federal)",18.80342,16.23932,15.38462,14.24501,13.39031,13.67521,8.262108
Health department (non-clinic),18.89764,15.74803,15.74803,12.59843,12.59843,11.81102,12.598425
Health department clinic/public health clinic,19.60784,17.32026,15.19608,13.56209,14.05229,16.50327,3.758170
LBGTQ Health Center/Community Center,19.04762,16.07143,14.88095,13.09524,14.88095,12.50000,9.523810


# MPOX Data

## Offering MPOX vax counts

In [226]:
mpox_vax_counts <- table(final_data$Q261)
mpox_vax_counts


 No Yes 
405 565 

## Offering MPOX vax perc

In [227]:
mpox_vax_perc <- prop.table(mpox_vax_counts)*100
mpox_vax_perc


      No      Yes 
41.75258 58.24742 

## MPOX vax offering by organization type counts

In [228]:
# Calculate the count of "Yes" and "No" responses by site for column "Q261"
mpox_vax_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes = sum(Q261 == "Yes", na.rm = TRUE),
    No = sum(Q261 == "No", na.rm = TRUE)
  )

# Print the counts of "Yes," "No," and "Unsure" responses by site for column "Q261"
mpox_vax_counts_by_site

Q1,Yes,No
<chr>,<int>,<int>
Academic hospital/clinic setting,48,24
Advocacy organization,0,3
AIDS Service Organization (ASO),64,51
Community Based Organization (CBO),45,70
Federally Qualified Health Center (FQHC),77,46
"Government entity (Local, State, Federal)",68,22
Health department (non-clinic),31,6
Health department clinic/public health clinic,110,19
LBGTQ Health Center/Community Center,26,18


## MPOX vax offering by organization type percentages

In [229]:
# Calculate the percentage of "Yes" and "No" responses for each site (column "Q1")
mpox_vax_perc_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes_Percentage = sum(Q261 == "Yes", na.rm = TRUE) / n() * 100,
    No_Percentage = sum(Q261 == "No", na.rm = TRUE) / n() * 100
  )

# Print the percentage of "Yes" and "No" responses for each site
mpox_vax_perc_by_site

Q1,Yes_Percentage,No_Percentage
<chr>,<dbl>,<dbl>
Academic hospital/clinic setting,60.00000,30.00000
Advocacy organization,0.00000,60.00000
AIDS Service Organization (ASO),48.12030,38.34586
Community Based Organization (CBO),34.88372,54.26357
Federally Qualified Health Center (FQHC),57.46269,34.32836
"Government entity (Local, State, Federal)",64.76190,20.95238
Health department (non-clinic),75.60976,14.63415
Health department clinic/public health clinic,75.34247,13.01370
LBGTQ Health Center/Community Center,47.27273,32.72727


## High MPOX vaccine uptake among LGBTQ clients/clients/patients counts

In [230]:
mpox_uptake_counts <- table(final_data$Q262)
mpox_uptake_counts


    No Unsure    Yes 
   109    157    300 

## High MPOX vaccine uptake among LGBTQ clients/clients/patients percentages

In [231]:
mpox_uptake_perc <- prop.table(mpox_uptake_counts)*100
mpox_uptake_perc


      No   Unsure      Yes 
19.25795 27.73852 53.00353 

## High MPOX vaccine uptake among LGBTQ clients/clients/patients by organization type counts

In [232]:
# Calculate the count of "Yes", "No", and "Unsure" responses by site for column "Q262"
mpox_uptake_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes = sum(Q262 == "Yes", na.rm = TRUE),
    No = sum(Q262 == "No", na.rm = TRUE),
    Unsure = sum(Q262 == "Unsure", na.rm = TRUE)
  )

# Print the counts of "Yes", "No", and "Unsure" responses by site for column "Q262"
mpox_uptake_counts_by_site

Q1,Yes,No,Unsure
<chr>,<int>,<int>,<int>
Academic hospital/clinic setting,22,10,16
Advocacy organization,0,0,0
AIDS Service Organization (ASO),40,8,16
Community Based Organization (CBO),27,7,11
Federally Qualified Health Center (FQHC),41,14,22
"Government entity (Local, State, Federal)",31,17,20
Health department (non-clinic),13,8,10
Health department clinic/public health clinic,57,24,29
LBGTQ Health Center/Community Center,17,1,9


## High MPOX vaccine uptake among LGBTQ clients/clients/patients by organization type percentages

In [233]:
# Calculate the percentage of "Yes", "No", and unsure responses for each site 
mpox_uptake_perc_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes_Percentage = sum(Q262 == "Yes", na.rm = TRUE) / n() * 100,
    No_Percentage = sum(Q262 == "No", na.rm = TRUE) / n() * 100,
    Unsure_Percentage = sum(Q262 == "Unsure", na.rm = TRUE) / n() * 100
  )

# Print the percentage of "Yes", "No", and unsure responses for each site
mpox_uptake_perc_by_site

Q1,Yes_Percentage,No_Percentage,Unsure_Percentage
<chr>,<dbl>,<dbl>,<dbl>
Academic hospital/clinic setting,27.500000,12.500000,20.000000
Advocacy organization,0.000000,0.000000,0.000000
AIDS Service Organization (ASO),30.075188,6.015038,12.030075
Community Based Organization (CBO),20.930233,5.426357,8.527132
Federally Qualified Health Center (FQHC),30.597015,10.447761,16.417910
"Government entity (Local, State, Federal)",29.523810,16.190476,19.047619
Health department (non-clinic),31.707317,19.512195,24.390244
Health department clinic/public health clinic,39.041096,16.438356,19.863014
LBGTQ Health Center/Community Center,30.909091,1.818182,16.363636


## Reason for MPOX vaccine hesitancy or refusal counts

In [234]:
mpox_hesitancy_counts <- table(final_data$Q263)
mpox_hesitancy_counts


                    Concerns about vaccine pain/scarring 
                                                      34 
                           Concerns about vaccine safety 
                                                      73 
                              Cultural/religious reasons 
                                                       7 
Lack of knowledge about MPOX transmission and/or vaccine 
                                                     179 
                        Low assumption of infection risk 
                                                     128 
                                        Medical mistrust 
                                                      73 
                                       None of the above 
                                                      40 
                                                   Other 
                                                      31 

## Reason for MPOX vaccine hesitancy or refusal percentages

In [235]:
mpox_hesitancy_perc <- prop.table(mpox_hesitancy_counts)*100
mpox_hesitancy_perc


                    Concerns about vaccine pain/scarring 
                                                6.017699 
                           Concerns about vaccine safety 
                                               12.920354 
                              Cultural/religious reasons 
                                                1.238938 
Lack of knowledge about MPOX transmission and/or vaccine 
                                               31.681416 
                        Low assumption of infection risk 
                                               22.654867 
                                        Medical mistrust 
                                               12.920354 
                                       None of the above 
                                                7.079646 
                                                   Other 
                                                5.486726 

## Contracting MPOX concerns counts

In [236]:
mpox_concern_counts <- table(final_data$Q265)
mpox_concern_counts


    No Unsure    Yes 
   108    118    339 

## Contracting MPOX concerns percentages

In [237]:
mpox_concern_perc <- prop.table(mpox_concern_counts)*100
mpox_concern_perc


      No   Unsure      Yes 
19.11504 20.88496 60.00000 

## Awareness of HealthHIV MPOX resource center counts

In [238]:
mpox_resource_counts <- table(final_data$Q266)
mpox_resource_counts


 No Yes 
356 209 

## Awareness of HealthHIV MPOX resource center counts

In [239]:
mpox_resource_perc <- prop.table(mpox_resource_counts)*100
mpox_resource_perc


      No      Yes 
63.00885 36.99115 

# Burnout Data